In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

In [3]:
import tensorflow as tf

2023-07-04 15:08:41.645719: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-04 15:08:41.723235: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [4]:
from tqdm import tqdm

In [5]:
from gahyparopt.widgets import *
from gahyparopt.GAUtilities import *

In [6]:
mnist = load_data()

In [7]:
import glob

In [8]:
players = ["player_{0:02d}".format(i) for i in range(1,8+1)]#[fn.split(".")[0] for fn in glob.glob('*.json')]

In [9]:
players

['player_01',
 'player_02',
 'player_03',
 'player_04',
 'player_05',
 'player_06',
 'player_07',
 'player_08']

In [10]:
game = dict([(p, game_instance(
            ga=GADriver(
            layer_counts=HIDDEN_LAYER_COUNT,
            no_neurons=HIDDEN_LAYER_NEURONS,
            rates=HIDDEN_LAYER_RATE,
            activations=HIDDEN_LAYER_ACTIVATIONS,
            layer_types=HIDDEN_LAYER_TYPE,
            optimizers=MODEL_OPTIMIZER,
            number_of_epochs=MODEL_EPOCHS,
            steps_per_epoch=MODEL_STEPS_PER_EPOCH,
            population_size=1,
            best_candidates_count=BEST_CANDIDATES_COUNT,
            random_candidates_count=RANDOM_CANDIDATES_COUNT,
            optimizer_mutation_probability=OPTIMIZER_MUTATION_PROBABILITY,
            layer_mutation_probability=HIDDEN_LAYER_MUTATION_PROBABILITY,
        ))
    ) for p in players])

In [11]:
%%capture
tf.keras.backend.clear_session
# Init all players with data and model.
for player, game in tqdm(games.items()):
    game.name = player
    game.data = mnist
    game.individuum = create_start_individuum(game.ga)[0]
    game.individuum.id = player
    
    
    
    
    

2023-07-04 15:08:44.136818: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-04 15:08:47.843264: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9671 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1a:00.0, compute capability: 7.5
2023-07-04 15:08:47.844493: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9671 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1b:00.0, compute capability: 7.5
2023-07-04 15:08:47.845642: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Creat

In [16]:
generation = 0

In [21]:
# Training
def develop_generation(generation, local_only=True):
    for player, game in games.items():
        clear_keras_session()
        print("************ {} ******************".format(player))
        if not local_only:
            sync_remote_to_local(game.name)
        if generation > 0:
            games[player].individuum = read_chromosome(game.name)
            games[player].individuum.id = game.name

        try:
            games[player].history = train_individuum(game.ga, game.data, game.individuum)

        except:
            games[player].history = None

        if games[player].history is not None:
            try:
                    loss, accuracy = evaluate_model(game.individuum.ml_model, game.data['x_val'], y=game.data['y_val'])
            except:
                accuracy = 0.0
                loss = 100.0

        else:
            accuracy = 0.0
            loss = 100.0

        games[player].individuum.loss = loss
        games[player].individuum.accuracy = accuracy

        write_chromosome(game.name, game.individuum)
        
        if not local_only:
            sync_local_to_remote(game.name)

    return (generation + 1)

    

In [32]:
generation = develop_generation(generation)

************ player_01 ******************
Epoch 1/8
100/100 [==============================] - 4s 29ms/step - loss: 0.7206 - accuracy: 0.7859 - val_loss: 0.3586 - val_accuracy: 0.9002
Epoch 2/8
100/100 [==============================] - 3s 28ms/step - loss: 0.3851 - accuracy: 0.8891 - val_loss: 0.3057 - val_accuracy: 0.9099
Epoch 3/8
100/100 [==============================] - 3s 27ms/step - loss: 0.3420 - accuracy: 0.9018 - val_loss: 0.2799 - val_accuracy: 0.9189
Epoch 4/8
100/100 [==============================] - 3s 27ms/step - loss: 0.3139 - accuracy: 0.9084 - val_loss: 0.2707 - val_accuracy: 0.9197
Epoch 5/8
100/100 [==============================] - 3s 27ms/step - loss: 0.2991 - accuracy: 0.9129 - val_loss: 0.2571 - val_accuracy: 0.9251
Epoch 6/8
100/100 [==============================] - 3s 28ms/step - loss: 0.2860 - accuracy: 0.9163 - val_loss: 0.2463 - val_accuracy: 0.9281
Epoch 7/8
100/100 [==============================] - 3s 28ms/step - loss: 0.2740 - accuracy: 0.9203 - val_

In [16]:
generation

2

# Danger zone!!!

In [15]:
delete_local()
delete_remote()